In [ ]:
import os
import sys
import numpy as np
import scipy.sparse as sp
from tqdm import tqdm
from keras.layers import Input, Dropout
from keras.models import Model
from keras.optimizers import Adam
from keras.regularizers import l2

In [ ]:
# the path of execution
EXE_PATH = os.path.abspath(os.path.curdir)
# the path of the vendor files
VENDOR_PATH = os.path.join(EXE_PATH, 'vendor')
# the vendors to include in the system path
VENDORS = ['keras-gcn']
# create the absolute paths for all vendors
VENDORS = list(map(lambda x: os.path.join(VENDOR_PATH, x), VENDORS))
# update the Python path to include necessary vendor module
sys.path += VENDORS

In [ ]:
from kegra.layers.graph import GraphConvolution
from kegra.utils import load_data, preprocess_adj, get_splits, evaluate_preds
from ggcn import GaussianGraphConvolution, kl_reg

# Data

In [ ]:
X, A, y = load_data('data/cora/', dataset='cora')
X /= X.sum(1).reshape(-1, 1)
A = preprocess_adj(A)
y_train, y_val, y_test, idx_train, idx_val, idx_test, train_mask = get_splits(y)

In [ ]:
X

In [ ]:
A

In [ ]:
y

In [ ]:
support = 1
graph = [X, A]
G = [Input(shape=(None, None), batch_shape=(None, None), sparse=True)]

In [ ]:
X_in = Input(shape=(X.shape[1],))

# Model

## Training Loop

In [ ]:
def train(model, epochs=200, patience=10):
    """
    """
    wait = 0
    preds = None
    best_val_loss = 99999
    # Fit
    progress = tqdm(range(1, epochs + 1))
    for epoch in progress:
        # Single training iteration (we mask nodes without labels for loss calculation)
        model.fit(graph, y_train, sample_weight=train_mask,
                  batch_size=A.shape[0], epochs=1, shuffle=False, verbose=0)
        # Predict on full dataset
        preds = model.predict(graph, batch_size=A.shape[0])
        # Train / validation scores
        train_val_loss, train_val_acc = evaluate_preds(preds, [y_train, y_val], [idx_train, idx_val])
        # update the progress bar
        progress.set_postfix(
#             train_loss=train_val_loss[0],
            train_acc=train_val_acc[0],
#             val_loss=train_val_loss[1],
            val_acc=train_val_acc[1],
        )
        # Early stopping
        if train_val_loss[1] < best_val_loss:
            best_val_loss = train_val_loss[1]
            wait = 0
        else:
            if wait >= patience:
                break
            wait += 1
    # return test set evaluation metrics
    return evaluate_preds(preds, [y_test], [idx_test])

## GCN (baseline)

In [ ]:
# Define model architecture
# NOTE: We pass arguments for graph convolutional layers as a list of tensors.
# This is somewhat hacky, more elegant options would require rewriting the Layer base class.
H = Dropout(0.5)(X_in)
H = GraphConvolution(32, support, activation='relu', kernel_regularizer=l2(5e-4))([H]+G)
H = Dropout(0.5)(H)
Y = GraphConvolution(y.shape[1], support, activation='softmax')([H]+G)
model = Model(inputs=[X_in]+G, outputs=Y)
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01))
model.summary()

In [ ]:
train(model)[1]

## Gaussian Graph Convolution

In [ ]:
B1 = 5e-4
B2 = 5e-4

In [ ]:
H = Dropout(0.5)(X_in)
H1 = GaussianGraphConvolution(16,
    is_first=True,
    activation='relu',
    mean_regularizer=l2(B1),
    variance_regularizer=l2(B1),
#     dropout=0.5
)([H]+G)
Y = GaussianGraphConvolution(y.shape[1],
    is_last=True,
    activation='softmax',
)(H1+G)
model = Model(inputs=[X_in]+G, outputs=Y)
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01))
model.add_loss(kl_reg(*H1, B2), H1)
model.summary()

In [ ]:
train(model)[1]